In [1]:
import os

from pyspark.sql.functions import expr

from sedona.spark import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,application_1691435962515_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
INPUT_PATH = "s3a://overturemaps-us-west-2/release/2023-07-26-alpha.0/"
OUTPUT_PATH = "s3://wherobots-examples/data/overturemaps-us-west-2/release/2023-07-26-alpha.0/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sedona = SedonaContext.create(spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define a function to convert the geometry column from WKB to Sedona geometry type

In [4]:
def geomTypeConverter(df, df_name):
    new_columns = []
    # Find the geometry column and convert it to a Sedona geometry type column
    for col_name in df.schema.names:
        if col_name == "geometry":
            new_columns.append("ST_GeomFromWKB(geometry) AS geometry")
        else:
            new_columns.append(col_name)
    cols =','.join(new_columns)
    df.createOrReplaceTempView(df_name)
    df = sedona.sql("SELECT " + cols + " FROM "+df_name)
    df.createOrReplaceTempView(df_name)
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define a function to repartition the data by GeoHash value

This will greatly accelerate the spatial queries on GeoParquet files. 

The chosen GeoHash length is 2.

| GeoHash length | Grid Area width x height |
|:--------------:|:------------------------:|
|        1       | ≤ 5,000km X 5,000 Km     |
|        2       | ≤ 1,250km X 625km        |
|        3       | ≤ 156km X 156km          |
|        4       | ≤ 39.1km X 19.5km        |
|        5       | ≤ 4.89km X 4.89km        |
|        6       | ≤ 1.22km X 0.61km        |
|        7       | ≤ 153m X 153m            |
|        8       | ≤ 38.2m X 19.1m          |
|        9       | ≤ 4.77m X 4.77m          |
|       10       | ≤ 1.19m X 0.596m         |
|       11       | ≤ 149mm X 149mm          |
|        12      | ≤ 37.2mm X 18.6mm        |

In [5]:
def reorgByGeoHash(df):
    return df.withColumn("geohash", expr("ST_GeoHash(geometry, 2)")).repartition("geohash")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Place Dataset 

In [6]:
tbl_name = "df_place"
tbl_name_s3 = "theme=places/type=place"
df = sedona.read.format("parquet").load(INPUT_PATH + tbl_name_s3)
df = geomTypeConverter(df, tbl_name)
df = reorgByGeoHash(df)
df.printSchema()
df.write.partitionBy("geohash").mode("overwrite").format("geoparquet").save(OUTPUT_PATH + tbl_name_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- updatetime: string (nullable = true)
 |-- version: integer (nullable = true)
 |-- names: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- categories: struct (nullable = true)
 |    |-- main: string (nullable = true)
 |    |-- alternate: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- confidence: double (nullable = true)
 |-- websites: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- socials: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- emails: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- phones: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- brand: struct (nullable = true)
 |    |-- names: map (nulla

# Building Dataset 

In [7]:
tbl_name = "df_building"
tbl_name_s3 = "theme=buildings/type=building"
df = sedona.read.format("parquet").load(INPUT_PATH + tbl_name_s3)
df = geomTypeConverter(df, tbl_name)
df = reorgByGeoHash(df)
df.printSchema()
df.write.partitionBy("geohash").mode("overwrite").format("geoparquet").save(OUTPUT_PATH + tbl_name_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- updatetime: string (nullable = true)
 |-- version: integer (nullable = true)
 |-- names: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- level: integer (nullable = true)
 |-- height: double (nullable = true)
 |-- numfloors: integer (nullable = true)
 |-- class: string (nullable = true)
 |-- sources: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- bbox: struct (nullable = true)
 |    |-- minx: double (nullable = true)
 |    |-- maxx: double (nullable = true)
 |    |-- miny: double (nullable = true)
 |    |-- maxy: double (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- geohash: string (nullable = true)

# Admins Datasets

## Administrative Boundary Dataset

In [8]:
tbl_name = "df_admin"
tbl_name_s3 = "theme=admins/type=administrativeBoundary"
df = sedona.read.format("parquet").load(INPUT_PATH + tbl_name_s3)
df = geomTypeConverter(df, tbl_name)
df = reorgByGeoHash(df)
df.printSchema()
df.write.partitionBy("geohash").mode("overwrite").format("geoparquet").save(OUTPUT_PATH + tbl_name_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- updatetime: string (nullable = true)
 |-- version: integer (nullable = true)
 |-- names: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- adminlevel: integer (nullable = true)
 |-- maritime: string (nullable = true)
 |-- subtype: string (nullable = true)
 |-- localitytype: string (nullable = true)
 |-- context: string (nullable = true)
 |-- isocountrycodealpha2: string (nullable = true)
 |-- isosubcountrycode: string (nullable = true)
 |-- defaultlanugage: string (nullable = true)
 |-- drivingside: string (nullable = true)
 |-- sources: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- bbox: struct (nullable = true)
 |    |-- minx: double (nullable

## Locality Dataset

In [9]:
tbl_name = "df_locality"
tbl_name_s3 = "theme=admins/type=locality"
df = sedona.read.format("parquet").load(INPUT_PATH + tbl_name_s3)
df = geomTypeConverter(df, tbl_name)
df = reorgByGeoHash(df)
df.printSchema()
df.write.partitionBy("geohash").mode("overwrite").format("geoparquet").save(OUTPUT_PATH + tbl_name_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- updatetime: string (nullable = true)
 |-- version: integer (nullable = true)
 |-- names: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- adminlevel: integer (nullable = true)
 |-- maritime: string (nullable = true)
 |-- subtype: string (nullable = true)
 |-- localitytype: string (nullable = true)
 |-- context: string (nullable = true)
 |-- isocountrycodealpha2: string (nullable = true)
 |-- isosubcountrycode: string (nullable = true)
 |-- defaultlanugage: string (nullable = true)
 |-- drivingside: string (nullable = true)
 |-- sources: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- bbox: struct (nullable = true)
 |    |-- minx: double (nullable

# Transportation Datasets

## Connector Dataset

In [10]:
tbl_name = "df_connector"
tbl_name_s3 = "theme=transportation/type=connector"
df = sedona.read.format("parquet").load(INPUT_PATH + tbl_name_s3)
df = geomTypeConverter(df, tbl_name)
df = reorgByGeoHash(df)
df.printSchema()
df.write.partitionBy("geohash").mode("overwrite").format("geoparquet").save(OUTPUT_PATH + tbl_name_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- updatetime: timestamp (nullable = true)
 |-- version: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- subtype: string (nullable = true)
 |-- connectors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- road: string (nullable = true)
 |-- sources: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- bbox: struct (nullable = true)
 |    |-- minx: double (nullable = true)
 |    |-- maxx: double (nullable = true)
 |    |-- miny: double (nullable = true)
 |    |-- maxy: double (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- geohash: string (nullable = true)

## Segment Dataset

In [11]:
tbl_name = "df_segment"
tbl_name_s3 = "theme=transportation/type=segment"
df = sedona.read.format("parquet").load(INPUT_PATH + tbl_name_s3)
df = geomTypeConverter(df, tbl_name)
df = reorgByGeoHash(df)
df.printSchema()
df.write.partitionBy("geohash").mode("overwrite").format("geoparquet").save(OUTPUT_PATH + tbl_name_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- updatetime: timestamp (nullable = true)
 |-- version: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- subtype: string (nullable = true)
 |-- connectors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- road: string (nullable = true)
 |-- sources: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- bbox: struct (nullable = true)
 |    |-- minx: double (nullable = true)
 |    |-- maxx: double (nullable = true)
 |    |-- miny: double (nullable = true)
 |    |-- maxy: double (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- geohash: string (nullable = true)